# MTA Elevator and Escalator Status

The [Metropolitan Transportation Authority (MTA) Subway Elev & Esc](https://mta-elevators.herokuapp.com) Status brings users real time information about the service status of elevators and escalators throughout the five boroughs.

This quick and easy application allows users to select subway stations by borough which returns the number of escalators and color coded elevator service status at the station. The collapsible menu expands to display the elevator’s serving area, alternative route, bus connections and if out of service, provides the estimated return to service date and time. These features removes the stress of riders knowing the equipment status before arriving to the station. It also provides a general real time outage status every 5 minutes of all elevators and escalators throughout the city. 

The goal of this project is to provide reliable access information of subway elevators and escalators and improve the way riders access that information.

As we begin to welcome back ridership within the transit system, we must address accessibility concerns regarding capital projects and communicate clearly with riders adjustments needed to complete their journey.

The application data is collected from MTA’s open dataset and developed using HTML/CSS/JavaScript and Python technology.

## Get MTA API

- Get API key from https://api.mta.info/#/signup

## MTA Resources

- [Data Feeds](http://web.mta.info/developers/developer-data-terms.html#data)
- [Realtime Data Feeds](https://api.mta.info/#/landing)

In [1]:
#----------------------------------------------------------------------------#
# Imports
#----------------------------------------------------------------------------#
import requests, json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#----------------------------------------------------------------------------#
# Use API key to access MTA Data
#----------------------------------------------------------------------------#

from dotenv import load_dotenv, find_dotenv # imports module for dotenv
load_dotenv(find_dotenv()) # loads .env from root directory

# The root directory requires a .env file with API_KEY assigned/defined within
# and dotenv installed from pypi. Get API key from http://datamine.mta.info/user
# api_key = os.environ['API_KEY']

True

In [3]:
# Elevators feed
from urllib.request import Request, urlopen
from xml.etree.ElementTree import parse

In [4]:
# MTA NYC Transit Station Locations – Updated February 17, 2021
station_location = 'https://atisdata.s3.amazonaws.com/Station/Stations.csv'
station_location_data = pd.read_csv(station_location, header=0)

In [5]:
df = pd.DataFrame(station_location_data)
df

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,517,517,S15,SIR,Staten Island,Prince's Bay,SI,SIR,Open Cut,40.525507,-74.200064,St George,Tottenville,0,NaN
492,518,518,S14,SIR,Staten Island,Pleasant Plains,SI,SIR,Embankment,40.522410,-74.217847,St George,Tottenville,0,NaN
493,519,519,S13,SIR,Staten Island,Richmond Valley,SI,SIR,Open Cut,40.519631,-74.229141,St George,Tottenville,0,NaN
494,522,522,S09,SIR,Staten Island,Tottenville,SI,SIR,At Grade,40.512764,-74.251961,St George,NaN,1,NaN


In [6]:
df.dtypes

Station ID                 int64
Complex ID                 int64
GTFS Stop ID              object
Division                  object
Line                      object
Stop Name                 object
Borough                   object
Daytime Routes            object
Structure                 object
GTFS Latitude            float64
GTFS Longitude           float64
North Direction Label     object
South Direction Label     object
ADA                        int64
ADA Notes                 object
dtype: object

In [7]:
boros = df["Borough"].unique()
boros

array(['Q', 'M', 'Bk', 'Bx', 'SI'], dtype=object)

In [8]:
df['Borough Name'] = df['Borough'].replace(['Q', 'M', 'Bk', 'Bx', 'SI'],['Queens', 'Manhattan', 'Brooklyn', 'Bronx', 'Staten Island'])

In [9]:
df['Borough Name']

0             Queens
1             Queens
2             Queens
3             Queens
4             Queens
           ...      
491    Staten Island
492    Staten Island
493    Staten Island
494    Staten Island
495    Staten Island
Name: Borough Name, Length: 496, dtype: object

In [10]:
# Qeens Stations
queen_station = df[df["Borough Name"] == 'Queens'].copy()
queen_station

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Borough Name
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,Queens
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,Queens
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN,Queens
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN,Queens
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN,Queens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,461,461,718,IRT,Flushing,Queensboro Plaza,Q,7,Elevated,40.750582,-73.940202,Astoria - Flushing,Manhattan,0,NaN,Queens
462,461,461,R09,BMT,Astoria,Queensboro Plaza,Q,N W,Elevated,40.750582,-73.940202,Astoria - Flushing,Manhattan,0,NaN,Queens
463,462,606,719,IRT,Flushing,Court Sq,Q,7,Elevated,40.747023,-73.945264,Flushing,Manhattan,1,NaN,Queens
464,463,463,720,IRT,Flushing,Hunters Point Av,Q,7,Subway,40.742216,-73.948916,Flushing,Manhattan,0,NaN,Queens


In [11]:
# Brooklyn Stations
brooklyn_station = df[df["Borough Name"] == 'Brooklyn'].copy()
brooklyn_station

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Borough Name
23,24,620,R28,BMT,Broadway,Court St,Bk,R,Subway,40.694100,-73.991777,Manhattan,Bay Ridge - 95 St,0,NaN,Brooklyn
24,25,636,R29,BMT,Broadway,Jay St-MetroTech,Bk,R,Subway,40.692180,-73.985942,Manhattan,Bay Ridge - 95 St,1,NaN,Brooklyn
25,26,26,R30,BMT,Broadway - Brighton,DeKalb Av,Bk,B Q R,Subway,40.690635,-73.981824,Manhattan,Coney Island - Bay Ridge,1,NaN,Brooklyn
26,27,617,R31,BMT,4th Av,Atlantic Av-Barclays Ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN,Brooklyn
27,28,28,R32,BMT,4th Av,Union St,Bk,R,Subway,40.677316,-73.983110,Manhattan,Bay Ridge - 95 St,0,NaN,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,356,356,244,IRT,Nostrand,Church Av,Bk,2 5,Subway,40.650843,-73.949575,Manhattan,Flatbush,1,NaN,Brooklyn
357,357,357,245,IRT,Nostrand,Beverly Rd,Bk,2 5,Subway,40.645098,-73.948959,Manhattan,Flatbush,0,NaN,Brooklyn
358,358,358,246,IRT,Nostrand,Newkirk Av,Bk,2 5,Subway,40.639967,-73.948411,Manhattan,Flatbush,0,NaN,Brooklyn
359,359,359,247,IRT,Nostrand,Flatbush Av-Brooklyn College,Bk,2 5,Subway,40.632836,-73.947642,Manhattan,NaN,1,NaN,Brooklyn


In [12]:
# Manhattan Stations
manhattan_station = df[df["Borough Name"] == 'Manhattan'].copy()
manhattan_station

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Borough Name
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn,0,NaN,Manhattan
7,8,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn,0,NaN,Manhattan
8,9,9,R14,BMT,Broadway - Brighton,57 St-7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn,0,NaN,Manhattan
9,10,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn,2,Uptown only,Manhattan
10,11,611,R16,BMT,Broadway - Brighton,Times Sq-42 St,M,N Q R W,Subway,40.754672,-73.986754,Uptown & Queens,Downtown & Brooklyn,1,NaN,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,469,610,901,IRT,Lexington - Shuttle,Grand Central-42 St,M,S,Subway,40.752769,-73.979189,Times Sq,NaN,1,NaN,Manhattan
471,471,471,726,IRT,Flushing,34 St-Hudson Yards,M,7,Subway,40.755882,-74.001910,Queens,NaN,1,NaN,Manhattan
472,475,475,Q05,IND,Second Av,96 St,M,Q,Subway,40.784318,-73.947152,NaN,Downtown & Brooklyn,1,NaN,Manhattan
473,476,476,Q04,IND,Second Av,86 St,M,Q,Subway,40.777891,-73.951787,Uptown,Downtown & Brooklyn,1,NaN,Manhattan


In [13]:
# The Bronx Stations
bronx_station = df[df["Borough Name"] == 'Bronx'].copy()
bronx_station

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Borough Name
210,210,210,D01,IND,Concourse,Norwood-205 St,Bx,D,Subway,40.874811,-73.878855,NaN,Manhattan,0,NaN,Bronx
211,211,211,D03,IND,Concourse,Bedford Park Blvd,Bx,B D,Subway,40.873244,-73.887138,Norwood - 205 St,Manhattan,1,NaN,Bronx
212,212,212,D04,IND,Concourse,Kingsbridge Rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN,Bronx
213,213,213,D05,IND,Concourse,Fordham Rd,Bx,B D,Subway,40.861296,-73.897749,Bedford Pk Blvd & 205 St,Manhattan,0,NaN,Bronx
214,214,214,D06,IND,Concourse,182-183 Sts,Bx,B D,Subway,40.856093,-73.900741,Bedford Pk Blvd & 205 St,Manhattan,0,NaN,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,442,442,501,IRT,Dyre Av,Eastchester-Dyre Av,Bx,5,At Grade,40.888300,-73.830834,NaN,Manhattan,0,NaN,Bronx
443,443,443,502,IRT,Dyre Av,Baychester Av,Bx,5,Open Cut,40.878663,-73.838591,Eastchester - Dyre Av,Manhattan,0,NaN,Bronx
444,444,444,503,IRT,Dyre Av,Gun Hill Rd,Bx,5,Open Cut,40.869526,-73.846384,Eastchester - Dyre Av,Manhattan,1,NaN,Bronx
445,445,445,504,IRT,Dyre Av,Pelham Pkwy,Bx,5,Open Cut,40.858985,-73.855359,Eastchester - Dyre Av,Manhattan,0,NaN,Bronx


In [14]:
# Staten Island Stations
si_station = df[df["Borough Name"] == 'Staten Island'].copy()
si_station

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Borough Name
475,501,501,S31,SIR,Staten Island,St George,SI,SIR,Open Cut,40.643748,-74.073643,NaN,Tottenville,1,NaN,Staten Island
476,502,502,S30,SIR,Staten Island,Tompkinsville,SI,SIR,At Grade,40.636949,-74.074835,St George,Tottenville,0,NaN,Staten Island
477,503,503,S29,SIR,Staten Island,Stapleton,SI,SIR,Elevated,40.627915,-74.075162,St George,Tottenville,0,NaN,Staten Island
478,504,504,S28,SIR,Staten Island,Clifton,SI,SIR,Elevated,40.621319,-74.071402,St George,Tottenville,0,NaN,Staten Island
479,505,505,S27,SIR,Staten Island,Grasmere,SI,SIR,Open Cut,40.603117,-74.084087,St George,Tottenville,0,NaN,Staten Island
480,506,506,S26,SIR,Staten Island,Old Town,SI,SIR,Embankment,40.596612,-74.087368,St George,Tottenville,0,NaN,Staten Island
481,507,507,S25,SIR,Staten Island,Dongan Hills,SI,SIR,Embankment,40.588849,-74.096090,St George,Tottenville,1,NaN,Staten Island
482,508,508,S24,SIR,Staten Island,Jefferson Av,SI,SIR,Embankment,40.583591,-74.103338,St George,Tottenville,0,NaN,Staten Island
483,509,509,S23,SIR,Staten Island,Grant City,SI,SIR,Open Cut,40.578965,-74.109704,St George,Tottenville,0,NaN,Staten Island
484,510,510,S22,SIR,Staten Island,New Dorp,SI,SIR,Open Cut,40.573480,-74.117210,St George,Tottenville,0,NaN,Staten Island


In [15]:
# Elevator/Escalator Status Outage
outage_url = urlopen('http://web.mta.info/developers/data/nyct/nyct_ene.xml')
outage_xmldoc = parse(outage_url)